In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import numpy as np
import pandas as pd
import datetime
import seaborn as sns
import ogb
from tqdm import tqdm
import hiplot as hip
from copy import deepcopy

In [3]:
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch.utils.data import Subset

In [4]:
cwd = os.getcwd()
print(cwd)

/cluster/home/skyriakos/chemprop_run/git/notebooks


In [5]:
os.chdir('..')
import deepadr
from deepadr.dataset import *
from deepadr.utilities import *
from deepadr.run_workflow import *
from deepadr.chemfeatures import *
from deepadr.hyphelper import *
# from deepadr.model_gnn import GCN as testGCN
from deepadr.model_gnn_ogb import GNN, DeepAdr_SiameseTrf, ExpressionNN
# from deepadr.model_attn_siamese import *
from ogb.graphproppred import Evaluator
os.chdir(cwd)

In [6]:
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw

In [7]:
# from tdc.single_pred import Tox
# from tdc.multi_pred import DDI

In [8]:
rawdata_dir = '../data/raw/'
processed_dir = '../data/processed/'
up_dir = '..'

In [9]:
report_available_cuda_devices()

number of GPUs available: 8
cuda:0, name:GeForce GTX 1080 Ti
total memory available: 10.91650390625 GB
total memory allocated on device: 0.0 GB
max memory allocated on device: 0.0 GB
total memory cached on device: 0.0 GB
max memory cached  on device: 0.0 GB

cuda:1, name:GeForce GTX 1080 Ti
total memory available: 10.91650390625 GB
total memory allocated on device: 0.0 GB
max memory allocated on device: 0.0 GB
total memory cached on device: 0.0 GB
max memory cached  on device: 0.0 GB

cuda:2, name:GeForce GTX 1080 Ti
total memory available: 10.91650390625 GB
total memory allocated on device: 0.0 GB
max memory allocated on device: 0.0 GB
total memory cached on device: 0.0 GB
max memory cached  on device: 0.0 GB

cuda:3, name:GeForce GTX 1080 Ti
total memory available: 10.91650390625 GB
total memory allocated on device: 0.0 GB
max memory allocated on device: 0.0 GB
total memory cached on device: 0.0 GB
max memory cached  on device: 0.0 GB

cuda:4, name:GeForce GTX 1080 Ti
total memory av

In [10]:
n_gpu = torch.cuda.device_count()
n_gpu

8

In [11]:
device_cpu = get_device(to_gpu=False)
# device_gpu = get_device(True, index=0)

# fdtype = torch.float32

In [12]:
print("torch:", torch.__version__)
print("CUDA:", torch.version.cuda)

torch: 1.9.1
CUDA: 11.1


In [13]:
# print(torch.cuda.memory_summary(device=device_gpu))

### Preparing dataset 

In [14]:
# TDC Tox
DSdataset_name = 'MatchMakerDrugComb' #'OncoPolyPharmacology' #'DrugComb'

#fname_suffix = ds_config["fname_suffix"]
similarity_types = ['chem']
kernel_option = 'sqeuclidean'
data_fname = 'data_v1'
# interact_matfname = ds_config["interact_matfname"]
# exp_iden = 'simtypeall'
# ddi_interaction_labels_pth = ds_config["ddi_interaction_labels_pth"]

# up_dir, processed_dir, DSdataset_name, data_fname

In [15]:
targetdata_dir = create_directory(os.path.join(processed_dir, DSdataset_name, data_fname))
targetdata_dir_raw = create_directory(os.path.join(targetdata_dir, "raw"))
targetdata_dir_processed = create_directory(os.path.join(targetdata_dir, "processed"))
# # ReaderWriter.dump_data(dpartitions, os.path.join(targetdata_dir, 'data_partitions.pkl'))
print(targetdata_dir)

path_current_dir /cluster/home/skyriakos/chemprop_run/git/deepadr
path_current_dir /cluster/home/skyriakos/chemprop_run/git/deepadr
path_current_dir /cluster/home/skyriakos/chemprop_run/git/deepadr
/cluster/home/skyriakos/chemprop_run/git/data/processed/MatchMakerDrugComb/data_v1


In [16]:
%%time

# Make sure to first run the "data_generation" notebook first

dataset = MoleculeDataset(root=targetdata_dir)

CPU times: user 9.71 ms, sys: 464 ms, total: 473 ms
Wall time: 463 ms


In [17]:
print()
print(f'Dataset: {dataset}:')
print('====================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data0 = dataset[0]  # Get the first graph object.

# print()
# print(data)
# print('=============================================================')

# # Gather some statistics about the first graph.
# print(f'Number of nodes: {data.num_nodes}')
# print(f'Number of edges: {data.num_edges}')
# print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
# print(f'Contains isolated nodes: {data.contains_isolated_nodes()}')
# print(f'Contains self-loops: {data.contains_self_loops()}')
# print(f'Is undirected: {data.is_undirected()}')


Dataset: MoleculeDataset(96671):
Number of graphs: 96671
Number of features: 9
Number of classes: 2


In [18]:
data0

PairData(edge_index_a=[2, 68], x_a=[31, 9], edge_attr_a=[68, 3], edge_index_b=[2, 76], x_b=[35, 9], edge_attr_b=[76, 3], id=[1], y=[1], expression=[1, 972])

In [19]:
data0.expression.shape

torch.Size([1, 972])

In [20]:
data0.y

tensor([0], dtype=torch.int32)

In [21]:
len(dataset)

96671

In [22]:
# used_dataset = dataset

# If you want to use a smaller subset of the dataset for testing
smaller_dataset_len = int(len(dataset)/20)
used_dataset = dataset[:smaller_dataset_len]

In [23]:
fold_partitions = get_stratified_partitions(used_dataset.data.y[:smaller_dataset_len],
                                            num_folds=5, valid_set_portion=0.1, random_state=42)

fold_num: 0
train data
class: 0 norm count: 0.967519402127048
class: 1 norm count: 0.032480597872951995
validation data
class: 0 norm count: 0.9664082687338501
class: 1 norm count: 0.03359173126614987
test data
class: 0 norm count: 0.9669079627714581
class: 1 norm count: 0.033092037228541885

-------------------------
fold_num: 1
train data
class: 0 norm count: 0.967519402127048
class: 1 norm count: 0.032480597872951995
validation data
class: 0 norm count: 0.9664082687338501
class: 1 norm count: 0.03359173126614987
test data
class: 0 norm count: 0.9669079627714581
class: 1 norm count: 0.033092037228541885

-------------------------
fold_num: 2
train data
class: 0 norm count: 0.967519402127048
class: 1 norm count: 0.032480597872951995
validation data
class: 0 norm count: 0.9664082687338501
class: 1 norm count: 0.03359173126614987
test data
class: 0 norm count: 0.9669079627714581
class: 1 norm count: 0.033092037228541885

-------------------------
fold_num: 3
train data
class: 0 norm cou

In [24]:
# train_val_test_frac = [0.7, 0.1, 0.2]
# assert sum(train_val_test_frac) == 1

# torch.manual_seed(42)
# used_dataset = used_dataset.shuffle()

# num_train = round(train_val_test_frac[0] * len(used_dataset)) 
# num_trainval = round((train_val_test_frac[0]+train_val_test_frac[1]) * len(used_dataset))

# train_dataset = used_dataset[:num_train]
# val_dataset = used_dataset[num_train:num_trainval]
# test_dataset = used_dataset[num_trainval:]

In [25]:
# train_dataset = Subset(used_dataset, fold_partitions[0]['train'])
# val_dataset = Subset(used_dataset, fold_partitions[0]['validation'])
# test_dataset = Subset(used_dataset, fold_partitions[0]['test'])

In [26]:
# print(f'Number of training graphs: {len(train_dataset)}')
# print(f'Number of val graphs: {len(val_dataset)}')
# print(f'Number of test graphs: {len(test_dataset)}')

In [27]:
# print(dataset.num_classes)

In [28]:
# train_dataset[3]

In [29]:
# training params
tp = {
    "batch_size" : 512,
    "num_epochs" : 100,
    
    "emb_dim" : 300,
    "gnn_type" : "gatv2",
    "num_layer" : 5,
    "graph_pooling" : "mean", #attention
    
    "input_embed_dim" : None,
    "num_attn_heads" : 2,
    "num_transformer_units" : 1,
    "p_dropout" : 0.3,
#     "nonlin_func" : nn.ReLU(),
    "mlp_embed_factor" : 2,
    "pooling_mode" : 'attn',
    "dist_opt" : 'cosine',

    "base_lr" : 3e-5, #3e-4
    "l2_reg" : 1e-7,
    "loss_w" : 0.1,
    "margin_v" : 1.,

    "expression_dim" : 64,
    "expression_input_size" : 972,
    "exp_H1" : 500,
    "exp_H2" : 400
}

In [30]:
# input_embed_dim = [128]
num_attn_heads = [1,2,4]
# num_transformer_units = [1]
p_dropout = [0.3, 0.45]
# nonlin_func = [nn.ReLU()]
# mlp_embed_factor = [2]
# pooling_mode = ['attn']
# dist_opt = ['cosine']
l2_reg = [0,1e-06]
# batch_size = [400]
# num_epochs = [200]
loss_w = [0.05, 0.1]


In [31]:
hp_names = ["num_attn_heads",
            "p_dropout",
            "l2_reg",
            "loss_w"]

In [32]:
[globals()[i] for i in hp_names]

[[1, 2, 4], [0.3, 0.45], [0, 1e-06], [0.05, 0.1]]

In [33]:
hyperparam_space = list(itertools.product(*[globals()[i] for i in hp_names]))
print(len(hyperparam_space))

24


In [34]:
hyperparam_space[0]

(1, 0.3, 0, 0.05)

In [35]:
targetdata_dir_exp = create_directory(os.path.join(targetdata_dir, "experiments"))

path_current_dir /cluster/home/skyriakos/chemprop_run/git/deepadr


In [36]:
def generate_tp_hp(tp, hp, hp_names):
    tphp=deepcopy(tp)
    for i,n in enumerate(hp_names):
#         print(n)
#         print(hp[i])
        tphp[n] = hp[i]
    return tphp

In [37]:
# device_gpu = get_device(True, index=0)
# exp_dir = create_directory(os.path.join(targetdata_dir_exp, "exp0"))
# tphp = generate_tp_hp(tp, hyperparam_space[0], hp_names)

In [38]:
# run_exp(tphp, device_gpu, exp_dir)

In [39]:
def spawn_q_process(q_process):
    print(">>> spawning hyperparam search process")
    q_process.start()
    
def join_q_process(q_process):
    q_process.join()
    print("<<< joined hyperparam search process")
    
def create_q_process(tp, device_gpu, exp_dir, partition, used_dataset): #
#     fold_gpu_map = {0:gpu_num}
    return mp.Process(target=deepadr.hyphelper.run_exp, args=(tp, device_gpu, exp_dir, partition, used_dataset)) #

In [ ]:
import torch.multiprocessing as mp
mp.set_start_method("spawn", force=True)

queue = mp.Queue()
q_processes = []

for q_i in range(min(n_gpu, len(hyperparam_space))):
    device_gpu = get_device(True, index=q_i)
    exp_dir = create_directory(os.path.join(targetdata_dir_exp, "exp"+str(q_i)))
    tphp = generate_tp_hp(tp, hyperparam_space[q_i], hp_names)
    
    q_process = create_q_process(tp=tp, device_gpu=device_gpu, exp_dir=exp_dir,
                                 partition=fold_partitions[0], used_dataset=None)
    q_processes.append(q_process)
    spawn_q_process(q_process)

spawned_processes = n_gpu
    
for q_i in range(len(hyperparam_space)):
    join_q_process(q_processes[q_i])
    released_gpu_num = queue.get()
    print("released_gpu_num:", released_gpu_num)
    if(spawned_processes < len(hyperparam_space)):
        device_gpu = get_device(True, index=q_i)
        exp_dir = create_directory(os.path.join(targetdata_dir_exp, "exp"+str(q_i)))
        tphp = generate_tp_hp(tp, hyperparam_space[q_i], hp_names)

        q_process = create_q_process(tp=tp, device_gpu=device_gpu, exp_dir=exp_dir,
                                 partition=fold_partitions[0], used_dataset=None)
        q_processes.append(q_process)
        spawn_q_process(q_process)
        spawned_processes = spawned_processes + 1

path_current_dir /cluster/home/skyriakos/chemprop_run/git/deepadr
>>> spawning hyperparam search process
path_current_dir /cluster/home/skyriakos/chemprop_run/git/deepadr
>>> spawning hyperparam search process
path_current_dir /cluster/home/skyriakos/chemprop_run/git/deepadr
>>> spawning hyperparam search process
path_current_dir /cluster/home/skyriakos/chemprop_run/git/deepadr
>>> spawning hyperparam search process
path_current_dir /cluster/home/skyriakos/chemprop_run/git/deepadr
>>> spawning hyperparam search process
path_current_dir /cluster/home/skyriakos/chemprop_run/git/deepadr
>>> spawning hyperparam search process
path_current_dir /cluster/home/skyriakos/chemprop_run/git/deepadr
>>> spawning hyperparam search process
path_current_dir /cluster/home/skyriakos/chemprop_run/git/deepadr
>>> spawning hyperparam search process


Process Process-2:
Traceback (most recent call last):
  File "/opt/conda/envs/graphnn/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/opt/conda/envs/graphnn/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/cluster/home/skyriakos/chemprop_run/git/deepadr/hyphelper.py", line 93, in run_exp
    num_iter = len(train_loader)  # num_train_samples/batch_size
NameError: name 'train_loader' is not defined


gpu:  cuda:1
updated
num classes: 2


Process Process-3:
Traceback (most recent call last):
  File "/opt/conda/envs/graphnn/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/opt/conda/envs/graphnn/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/cluster/home/skyriakos/chemprop_run/git/deepadr/hyphelper.py", line 93, in run_exp
    num_iter = len(train_loader)  # num_train_samples/batch_size
NameError: name 'train_loader' is not defined


gpu:  cuda:2
updated
num classes: 2


Process Process-5:
Traceback (most recent call last):
  File "/opt/conda/envs/graphnn/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/opt/conda/envs/graphnn/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/cluster/home/skyriakos/chemprop_run/git/deepadr/hyphelper.py", line 93, in run_exp
    num_iter = len(train_loader)  # num_train_samples/batch_size
NameError: name 'train_loader' is not defined
Process Process-1:
Traceback (most recent call last):
  File "/opt/conda/envs/graphnn/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/opt/conda/envs/graphnn/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/cluster/home/skyriakos/chemprop_run/git/deepadr/hyphelper.py", line 93, in run_exp
    num_iter = len(train_loader)  # num_train_samples/batch_size
NameError: name 'train_loade

gpu:  cuda:4
updated
num classes: 2
gpu:  cuda:0
updated
num classes: 2
gpu:  cuda:5
updated
num classes: 2
gpu:  cuda:6
updated
num classes: 2
gpu:  cuda:7
updated
num classes: 2
gpu:  cuda:3
updated
num classes: 2
<<< joined hyperparam search process
